<img src="nova.jpeg" alt="logo" width="300" align="right" style="padding-right:90px;padding-top:1cm"/>

<p style="text-align:center; padding-right:90px; font-size:280%;line-height: 1.2;margin-top: 1cm">CNN Brain Tumor Identification using T1-Weighted            MRI Scans</p>

<p style="text-align:center; padding-right:90px; font-size:100%;line-height: 1;margin-bottom: 1cm">Miguel Lopes, Tiago Coelho </p>

## Problem Description

<p style="text-align:justify; padding-right:90px;">In this project, we will work towards developing a tumor segmentation tool, resorting to the use of Convolutional Neural Networks. Our algorithm aims to identify the tumor area, returning a mask that fully and accurately represents only the tumor, when imputed with a magnetic resonance image (MRI). The dataset that we will use contains three types of tumor(meningioma, glioma, or pituitary tumor), however, we will study these independently of their label (such information may be used posterior to results in order to find what type of tumor that is harder to identify and act accordingly). The goal is that the tool is able to receive an image from any given slice, and identify it accordingly. However, changes may be performed so the algorithm is “slice aware” if the results are poor.</p>

<p style="text-align:justify; padding-right:90px;">
    <b>Firstly lets import tesorflow and check for available GPUs. In deeplearnign, Linking a GPU is very important because there are a lot of iterations and it helps the CPU. Training the model will be much faster.
    </b>
</p>

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num GPUs Available:  1


2022-12-17 16:03:19.034005: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-17 16:03:19.034099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18186333277642422597
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 locality {
   bus_id: 1
 }
 incarnation: 15463268103206366825
 physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
 xla_global_id: -1]

Metal device set to: Apple M1 Pro


__GPU identified__ 

### Now lets run VGG16
<p style="text-align:justify; padding-right:90px;">
VGG-16 is a convolutional neural network that is 16 layers deep. The ImageNet database contains a pretrained version of the network that has been trained on more than a million images . The pretrained network can categorize photos into 1000 different object categories, including several animals, a keyboard, a mouse, or a pencil. The network has therefore acquired rich feature representations for a variety of images.
</p>

<img src="VGG16.jpeg" alt="logo" width="600" align="center" style="padding-right:90px;padding-top:1cm"/>

__Imports__

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, 
Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

<p style="text-align:justify; padding-right:90px;">
As shown in the figure above, Convolutional Bock takes the image inputs and applies 2 filter layers. Each layer has 64 filters and an activation function at the end 'ReLU'. 
    
ReLU is mainly used because it is simple, fast, and most important, empirically seems to work well. It takes the outputs of the convolutions and applies the maximum value between them and zero. This turns every single negative value to zero while keeping the positives as they are. As an activation function, it adds complexity to the model, preventing it from being a simple combination of linear functions which is also a linear function. Most complex problems like ours cannot be solved by applying linear regressions to the data because the problem is simply not linear.
</p>

<img src="RELU.png" alt="logo" width="600" align="center" style="padding-right:90px;padding-top:1cm"/>

In [ ]:
def conv_block(inputs, num_filters):
       
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [13]:
run train.py

Train: 1380 - 1380
Valid: 459 - 459
Test: 459 - 459
Epoch 1/5
345/345 [==============================] - ETA: 0s - loss: 0.7790 - dice_coef: 0.2210 - iou: 0.1299 - recall_2: 0.7499 - precision_2: 0.1643
Epoch 1: val_loss improved from inf to 0.72731, saving model to files/model.h5
345/345 [==============================] - 161s 461ms/step - loss: 0.7790 - dice_coef: 0.2210 - iou: 0.1299 - recall_2: 0.7499 - precision_2: 0.1643 - val_loss: 0.7273 - val_dice_coef: 0.2728 - val_iou: 0.1613 - val_recall_2: 0.8273 - val_precision_2: 0.4159 - lr: 1.0000e-04
Epoch 2/5
345/345 [==============================] - ETA: 0s - loss: 0.5377 - dice_coef: 0.4623 - iou: 0.3114 - recall_2: 0.6820 - precision_2: 0.6665
Epoch 2: val_loss improved from 0.72731 to 0.49475, saving model to files/model.h5
345/345 [==============================] - 158s 459ms/step - loss: 0.5377 - dice_coef: 0.4623 - iou: 0.3114 - recall_2: 0.6820 - precision_2: 0.6665 - val_loss: 0.4947 - val_dice_coef: 0.5055 - val_iou: 0.348

In [2]:
run eval.py

2022-12-17 16:46:19.626893: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-17 16:46:19.626975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
  0%|                                                   | 0/459 [00:00<?, ?it/s]2022-12-17 16:46:20.804388: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1615.mat.png
1/1 [==============================] - 1s 524ms/step


  0%|                                           | 1/459 [00:00<04:41,  1.63it/s]

1
1615.mat.png
Bichos/1615.mat.png
Bichos/1615.mat.png
1
2281.mat.png
1/1 [==============================] - 0s 15ms/step


  0%|▏                                          | 2/459 [00:00<02:24,  3.16it/s]

...

971.mat.png
1/1 [==============================] - 0s 14ms/step
1
971.mat.png
Bichos/971.mat.png
Bichos/971.mat.png
1


 21%|████████▊                                 | 96/459 [00:10<00:37,  9.60it/s]

462.mat.png
1/1 [==============================] - 0s 15ms/step
1
462.mat.png
Bichos/462.mat.png
Bichos/462.mat.png
1


 21%|████████▊                                 | 96/459 [00:10<00:38,  9.35it/s]


KeyboardInterrupt: 

In [8]:
%tensorboard --callbacks 

UsageError: Line magic function `%tensorboard` not found.


In [14]:
%load_ext tensorboard
%tensorboard --logdir files

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 1452), started 0:42:14 ago. (Use '!kill 1452' to kill it.)